In [1]:
import numpy as np
import pandas as pd

### Import Data

In [3]:
event = pd.read_csv('events.csv')

In [4]:
session = pd.read_csv('sessions.csv')

In [5]:
session = session[session.is_developer==False]

In [6]:
event.head()

,app_id,session_id,event,event_timestamp,event_value,user_id_hash
0,4724682771660800,5558845121177764917,45,1542215397132,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,4724682771660800,5558845121177764917,45,1542215484895,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,4724682771660800,7689508378645584666,.m5100869650219008,1541124410372,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,4724682771660800,2201961907282901522,4,1543713091129,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4724682771660800,2201961907282901522,6,1543713093116,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [7]:
session.head()

,app_id,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,is_session,is_developer,...,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash
0,4724682771660800,5558845121177764917,1542215364580,Asia/Manila,28800000.0,25837591,1538874289458,False,True,False,...,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,4724682771660800,2201961907282901522,1543712977293,Asia/Manila,28800000.0,35050130,1538874289458,False,True,False,...,PH,00,quezon city,14.676041,121.043701,en_GB,Android OS,47,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,4724682771660800,18781111175537580,1539215568666,Asia/Manila,28800000.0,11343848,1538874289458,False,True,False,...,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,4724682771660800,1477540082628742048,1540120743010,Asia/Manila,28800000.0,13499724,1538874289458,False,True,False,...,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4724682771660800,8184875317380844086,1542671625528,Asia/Manila,28800000.0,32788010,1538874289458,False,True,False,...,PH,00,makati,14.554729,121.024445,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...


### Transform Timestamp

In [8]:
event['event_time'] = pd.to_datetime(event.event_timestamp, unit='ms')

In [9]:
event.event_time.min()

Timestamp('2018-10-01 07:00:04.526000')

In [10]:
event.event_time.max()

Timestamp('2018-12-14 23:59:59.977000')

In [11]:
session['start_time'] = pd.to_datetime(session.start_timestamp, unit='ms')

In [12]:
session.start_time.min()

Timestamp('2018-10-01 07:00:04.526000')

In [13]:
session.start_time.max()

Timestamp('2018-12-14 23:59:59.484000')

### Create Lable for train

In [14]:
# get user_list for train set (user are from oct.1 - Nov.30)
user_train = pd.DataFrame({'user_id_hash': session[session.start_time<'2018-12-01'].user_id_hash.unique()})

In [15]:
user_train.head()

,user_id_hash
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...


In [16]:
len(user_train)

620138

In [17]:
# label time Nov.17-Nov.30
df_train = event[(event.event_time<'2018-12-01') & (event.event_time>='2018-11-17')]

In [18]:
df_train.event_time.max()

Timestamp('2018-11-30 23:59:59.926000')

In [19]:
df_train.event_time.min()

Timestamp('2018-11-17 00:00:00.011000')

In [20]:
# filter out users that have purchase records from Nov.17 to Nov.30
purchased_user = df_train[df_train.event=='8'].user_id_hash.unique()

In [21]:
len(purchased_user)

11662

In [22]:
# create label to those users with 1.
label = np.ones_like(purchased_user)

In [23]:
purchase = pd.DataFrame({'user_id_hash': purchased_user, 'label': label})

In [24]:
purchase.head()

,user_id_hash,label
0,deaf54103e439789b069a95c4650dc6cdc24d6c9d700f1...,1
1,9f2baaf04b765e7ee6b0a92f6c3bfc5924ca8a1fcbd359...,1
2,280a7625f66e60896e558eaeee6b3eb0419c488eed15ef...,1
3,d05708ee901094cba0d6525f187275a17e027a99196885...,1
4,76080f4f796b19f81112129dfe3f70ab2e54a7ea931c67...,1


In [25]:
train_label = pd.merge(user_train, purchase, how='left', on='user_id_hash')

In [26]:
train_label = train_label.fillna(0)

In [27]:
train_label.label.value_counts()

0    608480
1     11658
Name: label, dtype: int64

In [27]:
train_label.to_csv('train_label_14days.csv', index=False)

### Create Label for validation

In [28]:
# get user_list for train set (user are from oct.15 - Dec.14)
user_train2 = pd.DataFrame({'user_id_hash': session[session.start_time>='2018-10-15'].user_id_hash.unique()})

In [29]:
user_train2.head()

,user_id_hash
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...


In [30]:
len(user_train2)

544296

In [31]:
# label time Dec.1-Dec.14
df_train2 = event[event.event_time>='2018-12-01']

In [32]:
df_train2.event_time.max()

Timestamp('2018-12-14 23:59:59.977000')

In [33]:
df_train2.event_time.min()

Timestamp('2018-12-01 00:00:00.044000')

In [34]:
# filter out users that have purchase records from Nov.17 to Nov.30
purchased_user2 = df_train2[df_train2.event=='8'].user_id_hash.unique()

In [35]:
len(purchased_user2)

6126

In [36]:
# create label to those users with 1.
label = np.ones_like(purchased_user2)

In [37]:
purchase2 = pd.DataFrame({'user_id_hash': purchased_user2, 'label': label})

In [38]:
purchase2.head()

,user_id_hash,label
0,280a7625f66e60896e558eaeee6b3eb0419c488eed15ef...,1
1,0e02d992cae31bf3cc09708fc70db2cf935288bf715aa7...,1
2,f885fd72310deaa1b7591bb3af277e632c364d36bdda5e...,1
3,5fb8780d18dfc69e2c968508d624828ecff7b504bf2719...,1
4,75ce9e8bf0517bc7dec67b715f3caf38871135c5cd462b...,1


In [39]:
valid_label = pd.merge(user_train2, purchase2, how='left', on='user_id_hash')

In [40]:
valid_label = valid_label.fillna(0)

In [41]:
valid_label.label.value_counts()

0    538170
1      6126
Name: label, dtype: int64

In [42]:
valid_label.to_csv('valid_label_14days.csv', index=False)

### Split Train tables: Session/ event/ message

In [42]:
# Oct.1-Nov.16
session_train = session[session.start_time<'2018-11-17']

In [43]:
session_train.start_time.min()

Timestamp('2018-10-01 07:00:04.526000')

In [44]:
session_train.start_time.max()

Timestamp('2018-11-16 23:59:59.140000')

In [45]:
event_train = event[event.event_time<'2018-11-17']

In [46]:
event_train.event_time.min()

Timestamp('2018-10-01 07:00:04.526000')

In [47]:
event_train.event_time.max()

Timestamp('2018-11-16 23:59:59.990000')

In [ ]:
session_train.to_csv('session_train_14days.csv', index=False)

In [67]:
event_train.to_csv('event_train_14days.csv', index=False)

### Split Validation tables: Session/ event/ message

In [48]:
# Oct.15-Nov.30
session_validation = session[(session.start_time<'2018-12-01') & (session.start_time>='2018-10-15')]

In [49]:
session_validation.start_time.min()

Timestamp('2018-10-15 00:00:01.909000')

In [50]:
session_validation.start_time.max()

Timestamp('2018-11-30 23:59:59.606000')

In [51]:
event_validation = event[(event.event_time<'2018-12-01') & (event.event_time>='2018-10-15')]

In [52]:
event_validation.event_time.min()

Timestamp('2018-10-15 00:00:00.080000')

In [53]:
event_validation.event_time.max()

Timestamp('2018-11-30 23:59:59.926000')

In [ ]:
session_validation.to_csv('session_validation_14days.csv', index=False)
event_validation.to_csv('event_validation_14days.csv', index=False)

### Split Test table: Session/event/message

In [54]:
# Oct.29 - Dec. 14th
session_test = session[session.start_time>='2018-10-29']

In [55]:
session_test.start_time.min()

Timestamp('2018-10-29 00:00:00.447000')

In [56]:
session_test.start_time.max()

Timestamp('2018-12-14 23:59:59.484000')

In [57]:
event_test = event[event.event_time>='2018-10-29']

In [58]:
event_test.event_time.min()

Timestamp('2018-10-29 00:00:00.004000')

In [59]:
event_test.event_time.max()

Timestamp('2018-12-14 23:59:59.977000')

In [ ]:
session_test.to_csv('session_test_14days.csv', index=False)

In [60]:
event_test.to_csv('event_test_14days.csv', index=False)

### Note

* Already exclude developer info in session dataset
* The userlists created by the unique user_id in session dataset.

### Some basic info in these datasets

In [62]:
len(session_train.user_id_hash.unique())

482961

In [63]:
len(session_test.user_id_hash.unique()) 

426086

In [64]:
len(event_train.user_id_hash.unique())  # might because there are some developers info

483087

In [65]:
len(event_test.user_id_hash.unique())

423420